In [1]:
%dependency /add com.microsoft.semantic-kernel:semantickernel-api:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-bom:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-parent:1.0.0-rc2
%dependency /add com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.0.0-rc2

%dependency /override com.azure:azure-core:1.45.1

%dependency /override com.google.code.gson:gson:2.8.9
%dependency /override com.google.protobuf:protobuf-java:3.23.2
%dependency /override com.google.errorprone:error_prone_annotations:2.18.0
%dependency /override com.google.guava:guava:32.0.1-android
%dependency /override org.slf4j:slf4j-api:2.0.12

%dependency /add io.qdrant:client:1.8.0


%dependency /resolve

Adding dependency com.microsoft.semantic-kernel:semantickernel-api:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-bom:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-parent:1.0.0-rc2
Adding dependency com.microsoft.semantic-kernel:semantickernel-aiservices-openai:1.0.0-rc2
Adding dependency override com.azure:azure-core:1.45.1
Adding dependency override com.google.code.gson:gson:2.8.9
Adding dependency override com.google.protobuf:protobuf-java:3.23.2
Adding dependency override com.google.errorprone:error_prone_annotations:2.18.0
Adding dependency override com.google.guava:guava:32.0.1-android
Adding dependency override org.slf4j:slf4j-api:2.0.12
Adding dependency io.qdrant:client:1.8.0
Solving dependencies
Found dependencies count: 64
Add to classpath: /home/vscode/.local/share/jupyter/kernels/rapaio-jupyter-kernel/ivy_cache/com.microsoft.semantic-kernel/semantickernel-api/jars/semantickernel-api-1.0.0-rc2.jar
Add to classpath: /home/

In [2]:
import com.azure.ai.openai.OpenAIAsyncClient;
import com.azure.ai.openai.OpenAIClientBuilder;
import com.azure.core.credential.AzureKeyCredential;
import com.azure.core.credential.KeyCredential;

In [3]:
import com.microsoft.semantickernel.Kernel;
import com.microsoft.semantickernel.Kernel.Builder;
import com.microsoft.semantickernel.exceptions.ConfigurationException;
import com.microsoft.semantickernel.orchestration.FunctionResult;
import com.microsoft.semantickernel.orchestration.PromptExecutionSettings;
import com.microsoft.semantickernel.plugin.KernelPluginFactory;
import com.microsoft.semantickernel.semanticfunctions.KernelFunction;
import com.microsoft.semantickernel.semanticfunctions.KernelFunctionArguments;
import com.microsoft.semantickernel.semanticfunctions.KernelFunctionFromPrompt;
import com.microsoft.semantickernel.services.textcompletion.TextGenerationService;

import com.microsoft.semantickernel.aiservices.openai.chatcompletion.OpenAIChatCompletion;
import com.microsoft.semantickernel.services.chatcompletion.ChatCompletionService;

In [4]:
import java.io.File;
import java.nio.file.Files;
import java.nio.file.Path;
import com.google.gson.Gson; 
import com.google.gson.GsonBuilder; 

In [5]:
public class KBInfo
{
    public String kb;
    public String content;


    public KBInfo(){} 


    public String getKb() { return kb; }
    public void setKb(String kb) { this.kb = kb; }
    public String getContent() { return content; }
    public void setContent(String content) { this.content = content; }
}

In [6]:
String AZURE_KEY = "Your Azure OpenAI Service API Key";
String AZURE_ENDPOINT = "Your Azure OpenAI Service Endpoint";

In [7]:
OpenAIAsyncClient client = new OpenAIClientBuilder()
        .credential(new AzureKeyCredential(AZURE_KEY))
        .endpoint(AZURE_ENDPOINT)
        .buildAsyncClient();

SLF4J(W): No SLF4J providers were found.
SLF4J(W): Defaulting to no-operation (NOP) logger implementation
SLF4J(W): See https://www.slf4j.org/codes.html#noProviders for further details.


In [8]:
ChatCompletionService chatGenerationService = OpenAIChatCompletion.builder()
                        .withOpenAIAsyncClient(client)
                        .withModelId("Your Azure OpenAI Service gpt-3.5-turbo-instruct deployment name")
                        .build();

In [9]:
var filePlugin =  KernelPluginFactory
                    .importPluginFromDirectory(
                        Path.of("./", "plugins"),
                        "FilePlugin",
                        null);

In [10]:
Kernel kernel = Kernel.builder()
    .withAIService(ChatCompletionService.class, chatGenerationService)
    .build();

In [11]:
List<KBInfo> kbList =  new ArrayList<KBInfo>();

In [12]:

GsonBuilder builder = new GsonBuilder(); 
builder.setPrettyPrinting();

com.google.gson.GsonBuilder@3dcbde32

In [13]:
var notesFolder = "data/notes/";
File folder = new File(notesFolder);
File[] listOfFiles = folder.listFiles();
for (File file : listOfFiles) {
    if (file.isFile()) {
        String filePath = notesFolder + file.getName();
        // directory.getAbsolutePath()
        File directory = new File(filePath);
        // System.out.println(directory.getAbsolutePath());
        Path fileName= Path.of(directory.getAbsolutePath());
        String str = Files.readString(fileName);

        var result = kernel
            .invokeAsync(filePlugin.get("Notes"))
            .withArguments(
                KernelFunctionArguments.builder()
                    .withVariable("input", str)
                    .build())
            .block();
            
        System.out.println(result.getResult());

        Gson gson = new Gson();
        KBInfo info = gson.fromJson(result.getResult().toString(), KBInfo.class);
        kbList.add(info);

    }
}

{"kb":"Introduction to machine learning","content":"The course on classical machine learning for beginners provides an introduction to machine learning. It covers the core concepts of machine learning, the history of ML, ML and fairness, regression ML techniques, classification ML techniques, clustering ML techniques, natural language processing ML techniques, time series forecasting ML techniques, reinforcement learning, and real-world applications for ML. The curriculum does not cover deep learning, neural networks, or AI. Machine learning automates the process of pattern-discovery and has applications in various fields such as healthcare, finance, and weather prediction. Understanding the basics of machine learning is becoming essential across domains due to its widespread adoption."}
{"kb": "History of machine learning", "content": "Machine learning and artificial intelligence have a rich history that dates back to the 18th century. Notable discoveries such as Bayes Theorem, Least 

In [14]:
import com.google.gson.reflect.TypeToken;

In [15]:
var transcriptsFolder = "data/transcripts/";
File folder = new File(transcriptsFolder);
File[] listOfFiles = folder.listFiles();
for (File file : listOfFiles) {
    if (file.isFile()) {
        String filePath = transcriptsFolder + file.getName();
        // directory.getAbsolutePath()
        File directory = new File(filePath);
        // System.out.println(directory.getAbsolutePath());
        Path fileName= Path.of(directory.getAbsolutePath());
        String str = Files.readString(fileName);

        var result = kernel
            .invokeAsync(filePlugin.get("Transcrips"))
            .withArguments(
                KernelFunctionArguments.builder()
                    .withVariable("input", str)
                    .build())
            .block();
            
        System.out.println(result.getResult());
        var kbContent = result.getResult().toString();

        // var gsonList = new Gson().fromJson(kbContent, JsonArray.class);


        Gson gson = new Gson();

        List<KBInfo> kbResult = gson.fromJson(kbContent, new TypeToken<List<KBInfo>>(){}.getType());

        //val kbResult = Gson().fromJson(json, Array<KBInfo>::class.java)

        for (KBInfo info : kbResult) {
                // System.out.printf(info.kb);
                kbList.add(info);
        }

        // System.out.println(String.valueOf(kbContent.indexOf("[")));
        

    }
}

[{"kb": "Introducing ML for Beginners", "content": "This course is based on the free open source 26-lesson\nml for beginners curriculum from\nMicrosoft."}, 
{"kb": "The difference between AI and ML", "content": "AI is a science of\ngetting machines to accomplish tasks\nthat typically require human level\nintelligence."}, 
{"kb": "What you'll learn in this course", "content": "you'll learn some Core\nConcepts of ml a bit of History\nstatistical techniques like regression\nclassification clustering and more"}, 
{"kb": "What you won't learn in this course", "content": "this course won't\ncover data science deep learning neural\nnetworks and AI techniques other than ml"}, 
{"kb": "Why study Machine Learning", "content": "Machine Learning is a\nHot Topic because it's solving complex\nreal-world problems in so many areas"}]
[
    {"kb": "Decide if AI is the right approach", "content": "traditional software is well suited to solve problems where the solution can be described as a formal set o

In [16]:

for (KBInfo info : kbList) {
    System.out.println(info.kb);
}

Introduction to machine learning
History of machine learning
# Techniques of Machine Learning
Introducing ML for Beginners
The difference between AI and ML
What you'll learn in this course
What you won't learn in this course
Why study Machine Learning
Decide if AI is the right approach
Collect and prepare data
Train your model
Evaluate your model
Tune the hyperparameters
Test the model in the real world
Alan Turing and the Turing test
The Dartmouth Summer Research Project on AI
The golden years of AI
The AI winter
Resurgence and fall of AI for expert systems
Growth in AI driven by more data and more powerful hardware
Big Data
Increased awareness of ethical and responsible AI


In [17]:
List<String> kbResultList =  new ArrayList<String>();

for (KBInfo info : kbList) {
    kbResultList.add(info.kb+"-"+info.content);
}

In [18]:

import com.azure.ai.openai.models.EmbeddingItem;
import com.azure.ai.openai.models.Embeddings;
import com.azure.ai.openai.models.EmbeddingsOptions;

In [19]:
EmbeddingsOptions embeddingsOptions = new EmbeddingsOptions(
    kbResultList);

var embeddings = client.getEmbeddings("Your Azure OpenAI Service text-ada-002 deployment name", embeddingsOptions);

In [20]:
// for(EmbeddingItem item : embeddings.block().getData()){
//     System.out.printf("Index: %d.%n", item.getPromptIndex());
//     for (Double embedding : item.getEmbedding()) {
//         System.out.printf("%f;", embedding);
//     }
// }


In [21]:
import io.qdrant.client.QdrantClient;
import io.qdrant.client.QdrantGrpcClient;
import io.qdrant.client.grpc.Collections.Distance;
import io.qdrant.client.grpc.Collections.VectorParams;
import java.util.concurrent.ExecutionException;

In [22]:
import io.qdrant.client.grpc.Collections.VectorParams;

In [23]:
import static io.qdrant.client.PointIdFactory.id;
import static io.qdrant.client.ValueFactory.value;
import static io.qdrant.client.VectorsFactory.vectors;

In [24]:
QdrantClient qdrantClient = 
  new QdrantClient(QdrantGrpcClient.newBuilder("localhost",6334,false).build());

In [25]:
qdrantClient.createCollectionAsync("kb_collection", VectorParams.newBuilder().setDistance(Distance.Cosine).setSize(1536).build()).get();

result: true
time: 0.949627227


In [26]:
import io.qdrant.client.grpc.Points.PointStruct;
import io.qdrant.client.grpc.Points.UpdateResult;
import static io.qdrant.client.VectorsFactory.vectors;


In [27]:
import java.util.List;
import java.util.Map;

In [28]:
List<List<Float>> points =  new ArrayList<List<Float>>();

for(EmbeddingItem item : embeddings.block().getData()){
    System.out.printf("Index: %d.%n", item.getPromptIndex());
    List<Float> values = new ArrayList<Float>();
    for (Double embedding : item.getEmbedding()) {
        values.add(embedding.floatValue());
    }
    points.add(values);
}

Index: 0.
Index: 1.
Index: 2.
Index: 3.
Index: 4.
Index: 5.
Index: 6.
Index: 7.
Index: 8.
Index: 9.
Index: 10.
Index: 11.
Index: 12.
Index: 13.
Index: 14.
Index: 15.
Index: 16.
Index: 17.
Index: 18.
Index: 19.
Index: 20.
Index: 21.


In [31]:
kbResultList.get(0)

Introduction to machine learning-The course on classical machine learning for beginners provides an introduction to machine learning. It covers the core concepts of machine learning, the history of ML, ML and fairness, regression ML techniques, classification ML techniques, clustering ML techniques, natural language processing ML techniques, time series forecasting ML techniques, reinforcement learning, and real-world applications for ML. The curriculum does not cover deep learning, neural networks, or AI. Machine learning automates the process of pattern-discovery and has applications in various fields such as healthcare, finance, and weather prediction. Understanding the basics of machine learning is becoming essential across domains due to its widespread adoption.

In [33]:

List<PointStruct> tmp = new ArrayList<PointStruct>();
int i = 1;

for (List<Float> embedding : points) {

    var pp =  
            PointStruct.newBuilder()
            .setId(id(i))
            .setVectors(vectors(embedding))
            .putAllPayload(Map.of("info", value(kbResultList.get(i-1))))
            .build();

    i += 1;

    tmp.add(pp);

    

}

In [34]:
var updateResult = qdrantClient.upsertAsync("kb_collection", tmp).get();

In [35]:
updateResult

operation_id: 0
status: Completed


In [36]:
EmbeddingsOptions qembeddingsOptions = new EmbeddingsOptions(
    Arrays.asList("what is different from AI and ML"));

var qembeddings = client.getEmbeddings("Your Azure OpenAI Service text-ada-002 deployment name", qembeddingsOptions);

In [37]:
List<Float> qe = new ArrayList<Float>();


for(EmbeddingItem item : qembeddings.block().getData()){
    // System.out.printf("Index: %d.%n", item.getPromptIndex());
    for (Double embedding : item.getEmbedding()) {
        // System.out.printf("%f;", embedding);
        qe.add(embedding.floatValue());
    }
}

In [38]:
import io.qdrant.client.grpc.Points.ScoredPoint;

import io.qdrant.client.grpc.Points.SearchPoints;
import static io.qdrant.client.WithPayloadSelectorFactory.enable;

In [39]:
var  answer =
qdrantClient
        .searchAsync(
            SearchPoints.newBuilder()
                .setCollectionName("kb_collection")
                .addAllVector(qe)
                .setWithPayload(enable(true))
                .setLimit(1)
                .build()).get();

In [40]:
answer

[id {
  num: 5
}
payload {
  key: "info"
  value {
    string_value: "The difference between AI and ML-AI is a science of\ngetting machines to accomplish tasks\nthat typically require human level\nintelligence."
  }
}
score: 0.89992046
]